In [57]:
import os
import sys
from pathlib import Path

# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv()
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings(model="embedding-passage")

DB_URI = os.environ["POSTGRES_URI"]     
# from langchain_postgres import PGEngine, PGVectorStore
# pg_engine = PGEngine.from_connection_string(
#     url=DB_URI
# )

from langchain_postgres import PGVector
from langchain_core.documents import Document
vector_store = PGVector(
    embeddings=embeddings,
    collection_name="SPRI_TAG",
    connection=DB_URI
)

In [58]:
import pickle
papers = []
for path in sorted(Path(project_root).joinpath("outputs").glob("*_split_documents.pkl")):
    
    with open(path, "rb") as f:
        split_documents = pickle.load(f)
        for doc in split_documents:
            name = path.stem.replace("_output_split_documents", "")
            doc.metadata["title"] = name
           
            if name in ["SPRI_2022", "SPRI_2023", "SPRI_2025"]:
                doc.metadata["category"] = "AI-index"
                if name == "SPRI_2022":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI윤리", "AI인재"]
                elif name == "SPRI_2023":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI규제", "AI인재"]
                elif name == "SPRI_2025":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI스타트업", "정부정책"]
            elif "SPRI_is" in name:
                doc.metadata["category"] = "issue-report"
                if name == "SPRI_is_211":
                    doc.metadata["tag"] = ["AI스타트업", "기술사업화", "기업가정신", "AI인재"]
                elif name == "SPRI_is_212":
                    doc.metadata["tag"] = ["SW안전", "기능안전", "오토파일럿"]
                elif name == "SPRI_is_213":
                    doc.metadata["tag"] = ["미래기술", "산업전망", "AI스타트업", "AI인재"]
            elif "SPRI_c" in name:
                doc.metadata["category"] = "column"
                if name == "SPRI_c_1":
                    doc.metadata["tag"] = ["Govtech", "정부혁신", "디지털정부"]
                elif name == "SPRI_c_2":
                    doc.metadata["tag"] = ["기능안전", "산업전망", "AI인재"]
                elif name == "SPRI_c_3":
                    doc.metadata["tag"] = ["산업전망망", "기업가정신", "AI윤리", "AI인재"]
                
                    
    papers.append(split_documents)


In [59]:
papers[3][0].metadata

{'page': 1,
 'image_id': [4],
 'image_path': ['/images/SPRI_c_1_cropped_figure_4.png'],
 'text_summary': '- 주제: GovTech 확산의 두 가지 엔진은 정당성(Legitimacy)과 역능적 행위자성(Empowered Actorhood)이다. 글쓴이는 유두호로 SPRI의 연구원이다.\n\n- GovTech 정의 및 목적: GovTech는 정부(Government)와 기술(Technology)의 결합으로, 공공 서비스 개선과 정부 운영의 효율성 향상을 넘어서 사회 문제 해결 중심의 공공 혁신을 지향한다. 기술 자체보다 이를 행정에 어떻게 접목하고 민간과 어떻게 협업할 것인가에 방점이 찍혀 있으며, 새로운 형태의 거버넌스로 주목받는다.\n\n- 핵심 차이점: 기존 전자정부의 디지털화에 머물지 않고, 사회 문제 해결을 위한 혁신 주도와 민간 협업을 강조한다.\n\n- 시장 전망: 글로벌 GovTech 시장 규모는 2024년 약 6,155.9억 달러로 추정되며, 2033년에는 약 2조 3,050억 달러에 이를 것으로 전망된다. 2025년부터 2033년까지의 CAGR도 상당한 수준으로 예상된다(본문에 제시되나 구체 수치는 생략 가능).\n\n- 참고: 본문은 GovTech의 정의와 시장 규모 추정을 제시하며, 관련 출처로 여러 연구를 인용하고 있다.',
 'image_summary': '다음과 같이 한국어로 묘사할 수 있습니다.\n\n- 그림은 GovTech의 디지털 공공 서비스 혁신을 상징하는 일러스트로 보인다.\n- 중앙에 큰 스마트폰/패널 모양의 기기가 배치되어 있고, 그 위아래로 세 명의 사람들이 서로 자료를 들고 대화하는 모습이다.\n  - 왼쪽 인물: 파란 재킷을 입은 남성이 노트를 들고 프레젠테이션을 하는 듯한 자세.\n  - 위 칸에 앉아 있는 인물: 노트를 들고 있는 남성의 모습으로, 기기 위에 올라앉아 있다.\n  - 오른쪽 인물: 노란 색 치마와 파란 셔츠를 입은 여

In [60]:
len(papers)

9

In [61]:
for paper in papers:
    vector_store.add_documents(paper)

In [55]:
papers[3][0].metadata

{'page': 1,
 'image_id': [4],
 'image_path': ['/images/SPRI_c_1_cropped_figure_4.png'],
 'text_summary': '- 주제: GovTech 확산의 두 가지 엔진은 정당성(Legitimacy)과 역능적 행위자성(Empowered Actorhood)이다. 글쓴이는 유두호로 SPRI의 연구원이다.\n\n- GovTech 정의 및 목적: GovTech는 정부(Government)와 기술(Technology)의 결합으로, 공공 서비스 개선과 정부 운영의 효율성 향상을 넘어서 사회 문제 해결 중심의 공공 혁신을 지향한다. 기술 자체보다 이를 행정에 어떻게 접목하고 민간과 어떻게 협업할 것인가에 방점이 찍혀 있으며, 새로운 형태의 거버넌스로 주목받는다.\n\n- 핵심 차이점: 기존 전자정부의 디지털화에 머물지 않고, 사회 문제 해결을 위한 혁신 주도와 민간 협업을 강조한다.\n\n- 시장 전망: 글로벌 GovTech 시장 규모는 2024년 약 6,155.9억 달러로 추정되며, 2033년에는 약 2조 3,050억 달러에 이를 것으로 전망된다. 2025년부터 2033년까지의 CAGR도 상당한 수준으로 예상된다(본문에 제시되나 구체 수치는 생략 가능).\n\n- 참고: 본문은 GovTech의 정의와 시장 규모 추정을 제시하며, 관련 출처로 여러 연구를 인용하고 있다.',
 'image_summary': '다음과 같이 한국어로 묘사할 수 있습니다.\n\n- 그림은 GovTech의 디지털 공공 서비스 혁신을 상징하는 일러스트로 보인다.\n- 중앙에 큰 스마트폰/패널 모양의 기기가 배치되어 있고, 그 위아래로 세 명의 사람들이 서로 자료를 들고 대화하는 모습이다.\n  - 왼쪽 인물: 파란 재킷을 입은 남성이 노트를 들고 프레젠테이션을 하는 듯한 자세.\n  - 위 칸에 앉아 있는 인물: 노트를 들고 있는 남성의 모습으로, 기기 위에 올라앉아 있다.\n  - 오른쪽 인물: 노란 색 치마와 파란 셔츠를 입은 여

In [ ]:
import pickle
papers = []
for path in sorted(Path(project_root).joinpath("outputs").glob("*_split_documents.pkl")):
    
    with open(path, "rb") as f:
        split_documents = pickle.load(f)
        for doc in split_documents:
            name = path.stem.replace("_output_split_documents", "")
            doc.metadata["title"] = name
           
            if name in ["SPRI_2022", "SPRI_2023", "SPRI_2025"]:
                doc.metadata["category"] = "AI-index"
                if name == "SPRI_2022":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI윤리", "AI인재"]
                elif name == "SPRI_2023":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI규제", "AI인재"]
                elif name == "SPRI_2025":
                    doc.metadata["tag"] = ["AI인덱스", "AI트렌드", "AI스타트업", "정부정책"]
            elif "SPRI_is" in name:
                doc.metadata["category"] = "issue-report"
                if name == "SPRI_is_211":
                    doc.metadata["tag"] = ["AI스타트업", "기술사업화", "기업가정신", "AI인재"]
                elif name == "SPRI_is_212":
                    doc.metadata["tag"] = ["SW안전", "기능안전", "오토파일럿"]
                elif name == "SPRI_is_213":
                    doc.metadata["tag"] = ["미래기술", "산업전망", "AI스타트업", "AI인재"]
            elif "SPRI_c" in name:
                doc.metadata["category"] = "column"
                if name == "SPRI_c_1":
                    doc.metadata["tag"] = ["Govtech", "정부혁신", "디지털정부"]
                elif name == "SPRI_c_2":
                    doc.metadata["tag"] = ["기능안전", "산업전망", "AI인재"]
                elif name == "SPRI_c_3":
                    doc.metadata["tag"] = ["산업전망", "기업가정신", "AI윤리", "AI인재"]
                
                    
    papers.append(split_documents)

In [142]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10, "filter": 
{"$and": [
            {"tag": {"$eq": "AI-index"}}
        ]}
})

In [143]:
retriever.invoke("AI 트렌드")

[]

In [51]:
from langchain_community.retrievers import BM25Retriever

In [9]:
docs = []
for paper in papers:
    docs.extend(paper)

In [10]:
len(docs)

247

In [127]:
from sqlalchemy import create_engine, text

engine = create_engine(DB_URI)
def sql_query() -> list[tuple[str, dict]]:
    with engine.connect() as conn:
        # Custom SQL query
        query = conn.execute(text(f"""
            SELECT 
                e.document,
                e.cmetadata            
            FROM langchain_pg_embedding e
            JOIN langchain_pg_collection c ON e.collection_id = c.uuid
            WHERE c.name = :collection_name
                AND e.cmetadata->>'title' = :title
                AND (e.cmetadata->'tag') ?| array[:tag1]
        """), {
            "collection_name": "SPRI_TAG",
            "title": "SPRI_2023",
            "tag1": "AI트렌드"
            
        })
        
        return query.fetchall()

In [128]:
result = sql_query()
arr = []
for r in result:
    arr.append(Document(page_content=r[0], metadata=r[1]))

In [129]:
arr =sorted(arr, key=lambda x: x.metadata["id"])

In [130]:
len(arr)

108

In [115]:
bm25_retriever = BM25Retriever.from_documents(arr)
bm25_retriever.k = 3

ValueError: not enough values to unpack (expected 3, got 0)

In [109]:
docs_sample = bm25_retriever.invoke("AI 트렌드")

In [54]:
retriever.invoke("AI 트렌드")

[Document(id='cbeef5ca-16a7-494b-a0ab-73468310d1f8', metadata={'id': 95, 'page': 54, 'title': 'SPRI_2023', 'image_id': [], 'image_path': [], 'text_summary': '요약: SPRi 이슈리포트 IS-159의 AI Index 2023 주요 내용과 시사점\n\n- 정부 전략과 이행 가속화\n  - 국가초고성능컴퓨팅 혁신전략(‘21.5.28), 국산 AI반도체를 활용한 K-클라우드 추진방안(‘22.12.12), 초거대 AI 경쟁력 강화방안(‘23.4.14) 등 정부 추진 전략의 단계적 이행 가속 필요.\n\n- AI 윤리와 거버넌스\n  - AI 오용 사례 증가와 생성 AI 확대로 AI 윤리에 대한 관심 증가.\n  - 공정성·편향 등 신뢰성 검증을 위한 구체적 기준과 가이드라인 필요.\n  - 활용 분야별 주요 지표 확립이 중요하며, 학습 데이터와 완화 방법에 따라 편향/독성의 영향이 달라질 수 있음.\n  - 편향성 및 독성 완화를 위한 명령 조정, 데이터 필터링 등 가이드라인 제시 필요.\n\n- 경제·투자 흐름\n  - 세계적 경기 침체의 영향으로 AI 분야 투자와 도입이 감소하는 가운데, 한국은 투자 확대를 통해 6위권 유지.\n  - 전 세계 투자 규모·건수·피투자기업 수가 감소하는 추세.\n  - 대형(10억 달러 이상) 투자를 제외한 규모별 투자 건수 감소로 신규 피투자 기업 수가 줄고 투자 집중화 심화.\n  - 미국의 영향력 여전히 높아 글로벌 AI 스타트업 투자의 약 52%를 차지하고 AI 채용 공고 비중도 높음.\n  - 중국은 AI 민간투자 규모와 AI 스타트업 수 등에서 2위 유지; 영국, 캐나다, 이스라엘, 독일 등도 상위 시장에 지속 포진.\n  - 한국은 근로자 AI 역량과 산업용 로봇 도입이 높고, 최근 AI 스타트업 관련 흐름도 주목.\n\n- 종합 시사점\n  - 정부의 전략 이행 가속과 함께 윤리 가이드라인 및 데

In [110]:
for doc in docs_sample:
    print(doc.metadata["tag"])
    
    print("-"*100)

['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
----------------------------------------------------------------------------------------------------
['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
----------------------------------------------------------------------------------------------------
['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
----------------------------------------------------------------------------------------------------
['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
----------------------------------------------------------------------------------------------------
['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
----------------------------------------------------------------------------------------------------
['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
----------------------------------------------------------------------------------------------------
['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
----------------------------------------------------------------------------------------------------
['AI인덱스', 'AI트렌드', 'AI규제', 'AI인재']
-------------